# item–item KNN CF using BM25‑weighted cosine + shrinkage, evaluated by leave‑one‑out.

1. Split by order time, not by (user, item) rows. Example: sort orders by order_timestamp, use first 80% of orders as train, last 20% as test (or a rolling window).
  a. Deduplicate within order: treat (order, item) as binary; handle quantities separately only if needed.
  
  b. Cold items (seen only in test) can’t be recommended; track coverage. Backfill with category/popularity.
  c. Build matrices on TRAIN ONLY


2. For each test order with ≥2 items, randomly hide 1 item and predict it from the remaining items in the same basket.
  a. Very popular items dominate cosine; use BM25/TF‑IDF or lift to reduce popularity bias.
  b. Shrinkage is essential for long‑tail items.
  c. For your “basket-aware” scenario, CF (item2vec/ALS) captures co-purchase complements well. But start with item-item CF (BM25-weighted cosine + shrinkage). Simple, strong, fast.
3. Use implicit feedback techniques (co‑occurrence, cosine/Jaccard/lift, BM25/TF‑IDF weighting, shrinkage) and add fallbacks (popularity, category‑aware) for baskets with too little context. Report HitRate@K / Recall@K / NDCG@K. (With one hidden item, HitRate@K = Recall@K.) To capture richer relations or to unify with next-order personalization later: train Item2vec or ALS; at serve time you can:
- do basket averaging (Item2vec) or fold-in (ALS) for context-aware recs, or
- precompute top-K neighbors per item from embeddings and serve like item-item CF.

If you split items from the same order across train and test, you’ll artificially inflate results.
Filters at serve time: availability, price range, category blacklists, etc.

for basket completion, both Item2vec and ALS work without historical user data by using the current cart as context. For home-page personalization with no context, you’ll need past interactions (ALS) or rely on non-CF signals (popularity/content).

In [213]:
import pandas as pd

cols = ['shopUserId', 'orderId', 'quantity', 'groupId', 'created']
tx = pd.read_csv('../../../data/processed//transactions_clean.csv', usecols=cols + ['status'], low_memory=False)
tx = tx[tx['status'] == 'active'][cols].astype({'quantity': int})

# --- remove top-occurring groupIds and save them separately ---
HEAD_FRAC = 0.01  # top 1% (tune) or set HEAD_TOPN instead
cnt = tx.groupby('groupId')['orderId'].nunique()  # occurrences by distinct orders
head_ids = cnt.sort_values(ascending=False).index[:max(5, int(len(cnt)*HEAD_FRAC))]

head_df = tx[tx['groupId'].isin(head_ids)].copy()     # saved popular items
tx = tx[~tx['groupId'].isin(head_ids)].copy()
tx

,shopUserId,orderId,quantity,groupId,created
0,812427,785001,1,261873,2025-08-05 20:14:28
1,831360,784985,4,261745,2025-08-05 19:55:36
8,230625,784973,1,260951,2025-08-05 19:39:49
9,230625,784973,1,590833,2025-08-05 19:39:49
10,230625,784973,1,260141,2025-08-05 19:39:49
...,...,...,...,...,...
250024,78202,158870,1,221416,2024-05-22 14:18:16
250026,78181,158841,1,265843,2024-05-22 13:42:39
250038,78145,158800,1,261518,2024-05-22 12:54:51
250039,78136,158791,1,542087,2024-05-22 12:44:01


In [142]:
head_ids

Index(['261637', '240187', '260646', '210338', '241562', '260596', '260513',
       '260695', '242024', '265298', '210186'],
      dtype='object', name='groupId')

### one row = one order, with items (list), quantities, created, n_items.

In [214]:
# 1) Parse timestamp
tx['created'] = pd.to_datetime(tx['created'], errors='coerce')  # add utc=True if you know tz

# 2) Drop exact duplicate rows
tx = tx.drop_duplicates(['orderId', 'groupId', 'created'])

# 3) Sort by order then timestamp; tie-break by groupId to keep it deterministic
tx = tx.sort_values(['orderId', 'created', 'groupId'])

baskets = (
    tx
      .groupby(['orderId', 'shopUserId'], sort=False)
      .agg(
          created=('created', 'min'),
          items=('groupId', list),
          quantities=('quantity', list),
          n_items=('groupId', 'size')
      )
      .reset_index()
)

In [215]:
#time-based split + dedupe items per order
baskets = baskets.sort_values(['created','orderId']).reset_index(drop=True)
split = int(len(baskets)*0.8)
train = baskets.iloc[:split].copy()
test  = baskets.iloc[split:].copy()

In [216]:
# enforce binary per order
train['items'] = train['items'].apply(lambda xs: sorted(set(xs)))
test['items']  = test['items'].apply(lambda xs: sorted(set(xs)))

# ignore quantities
train = train.drop(columns=['quantities'], errors='ignore')
test  = test.drop(columns=['quantities'], errors='ignore')


Produce co-occurrence counts to later convert into BM25/TF-IDF cosine, Jaccard, or lift, then apply shrinkage.

In [224]:
# next: co-occurrence counts (for item–item CF)
from collections import Counter
from itertools import combinations
co = Counter()
for items in train['items']:
    for a,b in combinations(items, 2):
        co[(a,b)] += 4 # of train orders containing both a and b

turn co into a weighted cosine similarity with lift, built on TRAIN-only item frequencies.

In [225]:
# Replace your cos/lift block with this BM25-weighted cosine + shrinkage
from collections import Counter, defaultdict
from itertools import combinations
import math

# binary popularity
ni = Counter(i for xs in train['items'] for i in xs)

ranked = [i for i, _ in ni.most_common()]        # global popularity order
head_skip = max(5, int(0.02 * len(ranked)))      # skip ultra-head (top ~2%)

bm25_pop = ranked[head_skip: head_skip + 1000]   # big cap, de-hot head

# same category popularity
PER_PREFIX_CAP = 20
bm25_pop_by_prefix = defaultdict(list)
for i in ranked[head_skip:]:
    p = str(i)[:2]
    if len(bm25_pop_by_prefix[p]) < PER_PREFIX_CAP:
        bm25_pop_by_prefix[p].append(i)

N  = len(train)


To de-bias popularity and basket-length effects: we BM25-weight item presence (via K1,B,idf), compute cosine in that weighted space (num/norm), and apply shrinkage by co-occurrence count (alpha) to down-weight noisy rare pairs.

K1 (≈0.8–1.6) —how fast extra occurrences stop adding weight (with binary baskets it mainly scales scores).
B (0–1) controls length normalization—how strongly you adjust for basket length (0 = none, 1 = full normalization).

In [259]:
K1, B = 1.2, 0.5
alpha = 50  

# IDF and avg doc length
idf = {i: max(0.0, math.log((N - df + 0.5) / (df + 0.5))) for i, df in ni.items()}
avgdl = (sum(len(set(xs)) for xs in train['items']) / N) if N else 0.0

# Accumulators
num = defaultdict(float)   # BM25-weighted dot products
norm = defaultdict(float)  # per-item squared norms
co   = Counter()           # binary co-occurrence counts (for shrinkage)

for xs in train['items']:
    items = sorted(set(xs))
    dl = len(items)
    denom = K1 * (1.0 - B + B * (dl / (avgdl + 1e-12)))
    # tf=1 for presence; BM25 weight per item in this order
    w = {i: idf.get(i, 0.0) * ((1.0 * (K1 + 1.0)) / (1.0 + denom)) for i in items}

    for i, wi in w.items():
        norm[i] += wi * wi
    for a, b in combinations(items, 2):
        wa, wb = w[a], w[b]
        if wa == 0.0 or wb == 0.0:
            continue
        num[(a, b)] += wa * wb
        num[(b, a)] += wa * wb
        co[(a, b)]  += 1
        co[(b, a)]  += 1

# Final similarities with significance shrinkage
sim = defaultdict(float)
for (i, j), n in num.items():
    s = n / (math.sqrt(norm[i]) * math.sqrt(norm[j]) + 1e-12)
    c = co[(i, j)]
    s *= (c / (c + alpha))          # shrinkage
    sim[(i, j)] = s

In [260]:
# Build per-item top BM25 neighbors (from sim) for scoring
from collections import defaultdict
bm25_nbrs = defaultdict(list)
for (i, j), s in sim.items():
    bm25_nbrs[i].append((j, s))
for i in list(bm25_nbrs):
    bm25_nbrs[i].sort(key=lambda x: x[1], reverse=True)
    bm25_nbrs[i] = [j for j, _ in bm25_nbrs[i][:50]]  # cap neighbors

# Popularity fallback (TRAIN)
bm25_pop = [i for i, _ in ni.most_common(50)]

eval_df is a filtered view of TEST: keep TEST orders with ≥2 items and add (context, target) by hiding one item.

In [261]:
# Prepare eval_df (binary, hide-1 on TEST)
import numpy as np

# binary per order
test['items'] = test['items'].apply(lambda xs: sorted(set(xs)))

# keep baskets with ≥2 items
eval_df = test[test['items'].str.len().ge(2)].copy()

# hide one item
rng = np.random.default_rng(42)
eval_df['target']  = eval_df['items'].apply(lambda xs: rng.choice(xs))
eval_df['context'] = eval_df.apply(lambda r: [i for i in r['items'] if i != r['target']], axis=1)

# Filter out rows where the hidden target is not in ni (i.e., not in train set)
eval_df = eval_df[eval_df['target'].apply(lambda t: t in ni)]

# Report n_eval and catalog coverage
n_eval = len(eval_df)
unique_targets = set(eval_df['target'])
catalog_covered = len(unique_targets)
catalog_total = len(ni)
catalog_coverage = catalog_covered / catalog_total if catalog_total > 0 else 0.0

print(f"n_eval: {n_eval}")
print(f"catalog coverage: {catalog_covered} of {catalog_total} ({catalog_coverage:.1%})")


n_eval: 4781
catalog coverage: 543 of 1020 (53.2%)


* For a basket’s current items (“context”), score other items by how similar they are to any item in the basket.
* Sum those scores, take the top-10 as recs, and if fewer than 10, fill with popular items.
* For evaluation, hide one item from each test basket and check if that hidden item appears in the top-10.
* The final number is the fraction of baskets where we “hit” the hidden item — **HitRate\@10**. 20% of recs had the actual item 

In [262]:
# Fixed-K=10 eval (hide-1)
def recommend10(ctx, per_anchor=10):
    scores = {}
    for a in ctx:
        for pair in bm25_nbrs.get(a, [])[:per_anchor]:
            if isinstance(pair, tuple):
                b, s = pair
            else:  # ID-only
                b, s = pair, sim.get((a, pair), 0.0)
            if b not in ctx:
                scores[b] = scores.get(b, 0.0) + s
    recs = [i for i, _ in sorted(scores.items(), key=lambda x: (-x[1], x[0]))[:10]]
    recs += [i for i in bm25_pop if i not in ctx and i not in recs]
    return recs[:10]


hits = 0
for _, r in eval_df.iterrows():
    hits += int(r['target'] in recommend10(r['context']))
hr10_cf = hits / len(eval_df)
hr10_cf


0.2568500313741895

In [263]:
# --- CF → category-pop only (no global pop); keep only truly similar CF (min_sim) ---

MIN_SIM = 1e-3  # set on TRAIN only (e.g., 0.01)
# assumes: bm25_nbrs, sim, bm25_pop_by_prefix (built on TRAIN), train, eval_df

def recommend10_hybrid(ctx, topk=10, per_anchor=None, min_sim=MIN_SIM):
    Z = max(1, len(ctx))
    scores, source = {}, {}

    # 1) CF scoring (only truly similar: s >= min_sim)
    for a in ctx:
        nbrs = bm25_nbrs.get(a, [])
        if per_anchor is not None:
            nbrs = nbrs[:per_anchor]
        for pair in nbrs:
            b, s = pair if isinstance(pair, tuple) else (pair, sim.get((a, pair), 0.0))
            if b in ctx or s < min_sim:
                continue
            scores[b] = scores.get(b, 0.0) + s / Z
            source.setdefault(b, "CF")

    # ranked CF list
    recs = [i for i, _ in sorted(scores.items(), key=lambda x: (-x[1], x[0]))]

    # 2) Fill only with category popularity (prefix buckets), no global fallback
    if len(recs) < topk:
        for p in dict.fromkeys(str(a)[:2] for a in ctx):  # prefixes from context
            for i in bm25_pop_by_prefix.get(p, []):
                if i in ctx or i in scores or i in recs:
                    continue
                recs.append(i)
                source.setdefault(i, "POP(cat)")
                if len(recs) >= topk:
                    break
            if len(recs) >= topk:
                break

    return recs[:topk], scores, source  # may be < topk if category list is short

# --- evaluation (HR@10; skips cold targets). Note: if recs < 10, it's still counted fairly. ---
train_items = set(i for xs in train['items'] for i in xs)
hits = n_eval = 0
for _, r in eval_df.iterrows():
    tgt = r['target']
    if tgt not in train_items:
        continue
    recs, _, _ = recommend10_hybrid(r['context'], topk=10, per_anchor=10)
    hits += int(tgt in recs)
    n_eval += 1

hr10_hybrid = hits / n_eval if n_eval else float('nan')
print({'HR@10_hybrid': hr10_hybrid})


{'HR@10_hybrid': 0.25664087011085546}


In [234]:
# --- one-time (before visualize_one) ---
import math
from collections import Counter
ni = Counter(i for xs in train['items'] for i in xs)   # TRAIN-only doc freq
_max = max(ni.values()) if ni else 1
pop_score = {i: math.log1p(ni[i]) / math.log1p(_max) for i in ni}  # ∈ [0,1]

# --- visualizer (aligned with recommend10_hybrid) ---
import numpy as np, pandas as pd

def visualize_one(r, topk=10, per_anchor=10, min_sim=MIN_SIM):
    ctx, tgt = r['context'], r['target']
    recs, scores, source = recommend10_hybrid(ctx, topk=topk, per_anchor=per_anchor, min_sim=min_sim)
    Z = max(1, len(ctx))
    rows = []
    for i in recs:
        if source.get(i) == "CF":
            contrib = [(a, sim.get((a,i), 0.0)/Z) for a in ctx if sim.get((a,i), 0.0) >= min_sim]
            ta = max(contrib, key=lambda x: x[1])[0] if contrib else ""
        else:
            ta = ""  # POP(cat)
        rows.append((i, scores.get(i, 0.0), ta, source.get(i, "POP(cat)"), pop_score.get(i, 0.0)))
    df = pd.DataFrame(rows, columns=["candidate","cf_score","top_anchor","source","pop_score"])
    df["hit"] = df["candidate"].eq(tgt)
    return df

# Example
rng = np.random.default_rng()
for idx in rng.choice(eval_df.index, size=min(5, len(eval_df)), replace=False):
    r = eval_df.loc[idx]
    print(f"\nORDER {r.get('orderId', idx)}  target={r['target']}  context={r['context']}")
    print(visualize_one(r).to_string(index=False))



ORDER 779641  target=292250  context=['281410', '291120', '291625', '292607']
candidate  cf_score top_anchor source  pop_score   hit
   292789  0.052289     291120     CF   0.948478 False
   292193  0.022584     291625     CF   0.850374 False
   290291  0.007280     291120     CF   0.856368 False
   290228  0.007118     291120     CF   0.865949 False
   291088  0.006192     291120     CF   0.901357 False
   291054  0.005961     291120     CF   0.885193 False
   290290  0.005374     291120     CF   0.853776 False
   294090  0.002361     291120     CF   0.931265 False
   200304  0.001958     281410     CF   0.999324 False
   291612  0.001569     291625     CF   0.564124 False

ORDER 741446  target=200304  context=['270600']
candidate  cf_score top_anchor   source  pop_score   hit
   270599  0.002415     270600       CF   0.743463 False
   210756  0.001759     270600       CF   0.844517 False
   270597  0.001681     270600       CF   0.787533 False
   270593  0.001425     270600       CF